## Prep Data

In [1]:
# MOUNTING GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os

wd = '/content/drive/MyDrive/Colab Notebooks/cs532_project'
print(os.listdir(wd))
os.chdir(wd)
print(os.getcwd())

['data']
/content/drive/MyDrive/Colab Notebooks/cs532_project


In [ ]:
check = '/content/drive/MyDrive/CS 532/cs532_project/data'
print(os.listdir(check))

['yelp_academic_dataset_review.json', 'yelp_academic_dataset_review.json.zip', 'review_raw.csv', 'review_raw_sorted.csv', 'review_raw_sorted_labels.csv', 'review_raw_sorted_labels_v2.csv', 'review_2005.csv', 'review_2005-06.csv', 'review_2005-07.csv', 'LR_model', 'review_2008.csv', 'LR_model_2', 'NB_model']


In [6]:
import os
import pandas as pd

# data_path = "/content/drive/MyDrive/CS 532/cs532_project/data"
data_path = "data"

test_data = pd.read_csv(os.path.join(data_path, "review_2005-07.csv"), header=0)
# test_data = pd.read_csv(os.path.join(data_path, "review_2008.csv"), header=0)
test_data.head(), test_data.shape

# test_data = spark.read.csv(, header=True)
# data = spark.createDataFrame(test_data)
# data.count(), data.columns

(                                                text  labels_3
 0  It's not chicago style pizza but it is still a...  positive
 1  Great service.  The people really want to help...  positive
 2  These guys really are the king of cheese steak...  positive
 3  Words cannot describe how much I miss this pla...  positive
 4  Food is decent but is a complete tourist trap....  negative,
 (20070, 2))

In [7]:
datadict = {"train": [], "test": []}
max_len = 0
for index, row in test_data.iterrows():
    item = {}
    item['text'] = row['text']
    max_len = max(max_len, len(row['text'].split(' ')))
    label = row['labels_3']
    item['label_v2'] = label
    item['label'] = {"positive": False, "negative": False, "neutral": False}
    item['label'][label] = True
    # print(item)
    if index < 16000:
      datadict["train"].append(item)
    else:
      datadict["test"].append(item)
    # break
len(datadict["train"]), len(datadict["test"]), max_len

(16000, 4070, 1031)

In [8]:
datadict["train"][0]

{'text': "It's not chicago style pizza but it is still awsome.  A chicago themed restaurant.  Owners used to live in Chi town and now are bringing the feel of the city to the West.  Beer selection rocks!",
 'label_v2': 'positive',
 'label': {'positive': True, 'negative': False, 'neutral': False}}

In [9]:
labels = ['positive', 'negative', 'neutral']
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels, id2label, label2id

(['positive', 'negative', 'neutral'],
 {0: 'positive', 1: 'negative', 2: 'neutral'},
 {'positive': 0, 'negative': 1, 'neutral': 2})

# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way. 

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

In [10]:
!pip install -q transformers==4.28.0 #datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.3 MB/s eta 0:00:00


## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [11]:
from transformers import AutoTokenizer
import numpy as np
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  label = examples["label_v2"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512) #max_len)
  # add labels
  if label == "positive": labels_matrix = np.array([1.0, 0.0, 0.0])
  elif label == "negative": labels_matrix = np.array([0.0, 1.0, 0.0])
  elif label == "neutral": labels_matrix = np.array([0.0, 0.0, 1.0])
  else:
    print('error')
  encoding["labels"] = torch.from_numpy(labels_matrix)

  encoding["input_ids"] = (torch.Tensor(encoding["input_ids"])).int()
  encoding["token_type_ids"] = (torch.Tensor(encoding["token_type_ids"])).int()
  encoding["attention_mask"] = (torch.Tensor(encoding["attention_mask"])).int()
  
  return encoding

In [12]:
examples = datadict['train'][0]
check = preprocess_data(examples)
check.keys(), type(check['input_ids'])

(dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels']),
 torch.Tensor)

In [13]:
encoded_dataset = {'train': [], 'test': []}
for type_, data_ in datadict.items():
  for data_2 in data_:
    # print(data_2)
    encoded_dataset[type_].append(preprocess_data(data_2))
  #   break
  # break
len(encoded_dataset['train']), len(encoded_dataset['test'])

(16000, 4070)

In [14]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [15]:
tokenizer.decode(example['input_ids'])

"[CLS] it's not chicago style pizza but it is still awsome. a chicago themed restaurant. owners used to live in chi town and now are bringing the feel of the city to the west. beer selection rocks! [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [16]:
example['labels']

tensor([1., 0., 0.], dtype=torch.float64)

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [17]:
len(labels), id2label

(3, {0: 'positive', 1: 'negative', 2: 'neutral'})

In [18]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [19]:
batch_size = 8
metric_name = "f1"

In [20]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.9 MB/s eta 0:00:00


In [27]:
from transformers import TrainingArguments, Trainer

batch_size = 16

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [28]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [23]:
encoded_dataset['train'][0]['labels'].type()

'torch.DoubleTensor'

In [ ]:
# encoded_dataset['train'][0]['input_ids']

In [24]:
#forward pass
outputs = model(input_ids=encoded_dataset['train'][0]['input_ids'].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7661, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[0.1171, 0.1831, 0.3324]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Let's start training!

In [29]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.302600,0.302902,0.795905,0.844226,0.773219


TrainOutput(global_step=1000, training_loss=0.3130090026855469, metrics={'train_runtime': 1583.8463, 'train_samples_per_second': 10.102, 'train_steps_per_second': 0.631, 'total_flos': 4209814683648000.0, 'train_loss': 0.3130090026855469, 'epoch': 1.0})

## Evaluate

After training, we evaluate our model on the validation set.

In [31]:
trainer.evaluate()

{'eval_loss': 0.3029022514820099,
 'eval_f1': 0.7959054720080879,
 'eval_roc_auc': 0.8442260442260443,
 'eval_accuracy': 0.7732186732186732,
 'eval_runtime': 136.0597,
 'eval_samples_per_second': 29.913,
 'eval_steps_per_second': 1.874,
 'epoch': 1.0}

## Inference

Let's test the model on a new sentence:

In [48]:
len(text.split(' '))

39

In [47]:
# text = "I'm happy I can finally train a model for multi-label classification"
# text = "I'm not sure how I feel about this place, but seems ok"
# text = "I don't like this place, it gets very cold"
text = "It's not chicago style pizza but it is still awsome.  A chicago themed restaurant.  Owners used to live in Chi town and now are bringing the feel of the city to the West.  Beer selection rocks!"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [49]:
logits = outputs.logits
logits.shape

torch.Size([1, 3])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [50]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['positive']


In [46]:
logits, predictions

(tensor([[-2.5799,  1.5948, -1.7209]], device='cuda:0',
        grad_fn=<AddmmBackward0>),
 array([0., 1., 0.]))